### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [6]:
import json
import nltk

In [7]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [8]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [9]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [11]:
print(docs[0])

[('李', 'O'), ('某', 'O'), ('，', 'O'), ('女', 'O'), ('，', 'O'), ('6', 'O'), ('7', 'O'), ('岁', 'O')]


In [17]:
#取2701筆
import random

rand_index = []
for x in range(3000):
    r = random.randint(0,48901)
    if len(rand_index)<2701 and r not in rand_index:
        rand_index.append(r)

In [18]:
docs2 = []
for idx in rand_index:
    docs2.append(docs[idx])

In [19]:
print(len(docs2))

2701


In [20]:
print(docs2[0])

[('诊', 'O'), ('其', 'O'), ('脉', 'O'), ('平', 'O'), ('，', 'O'), ('舌', 'O'), ('苔', 'O'), ('黄', 'O')]


In [21]:
data = []

for i, doc in enumerate(docs2):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [22]:
print(len(data))

2701


In [23]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [24]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
print(len(X_train))
print(len(X_test))

2160
541


In [26]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 14694
Seconds required: 0.187

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 16536.058630
Feature norm: 1.000000
Error norm: 4628.416725
Active features: 14614
Line search trials: 1
Line search step: 0.000005
Seconds required for this iteration: 0.059

***** Iteration #2 *****
Loss: 16433.294179
Feature norm: 1.005358
Error norm: 4428.598693
Active features: 13695
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #3 *****
Loss: 14990.699880
Feature norm: 1.203119
Error norm: 6161.373019
Active features: 9508
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

***** Iteration #42 *****
Loss: 2611.451285
Feature norm: 27.911292
Error norm: 1332.743506
Active features: 11534
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.031

***** Iteration #43 *****
Loss: 2517.640787
Feature norm: 28.945327
Error norm: 2222.026691
Active features: 11433
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.031

***** Iteration #44 *****
Loss: 2405.181991
Feature norm: 30.747279
Error norm: 1619.580029
Active features: 11337
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.031

***** Iteration #45 *****
Loss: 2321.598589
Feature norm: 31.970439
Error norm: 2140.169932
Active features: 11261
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.034

***** Iteration #46 *****
Loss: 2246.370694
Feature norm: 32.658779
Error norm: 586.170687
Active features: 11235
Line search trials: 1
Line search step: 1.000000
Secon

***** Iteration #85 *****
Loss: 1314.607050
Feature norm: 59.408504
Error norm: 136.611983
Active features: 7313
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #86 *****
Loss: 1312.287105
Feature norm: 59.866014
Error norm: 624.448787
Active features: 7230
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.031

***** Iteration #87 *****
Loss: 1309.327125
Feature norm: 60.018736
Error norm: 484.081514
Active features: 7216
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.034

***** Iteration #88 *****
Loss: 1305.456142
Feature norm: 60.343656
Error norm: 218.725022
Active features: 7227
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.039

***** Iteration #89 *****
Loss: 1303.141803
Feature norm: 60.474617
Error norm: 198.023034
Active features: 7205
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #131 *****
Loss: 1247.943726
Feature norm: 65.905073
Error norm: 59.377097
Active features: 6369
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #132 *****
Loss: 1247.053053
Feature norm: 65.982354
Error norm: 169.235979
Active features: 6323
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.031

***** Iteration #133 *****
Loss: 1246.115621
Feature norm: 65.976579
Error norm: 93.879170
Active features: 6318
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.034

***** Iteration #134 *****
Loss: 1245.183155
Feature norm: 66.042226
Error norm: 148.369588
Active features: 6272
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #135 *****
Loss: 1244.144062
Feature norm: 66.055076
Error norm: 98.702770
Active features: 6257
Line search trials: 1
Line search step: 1.000000
Seconds requ

***** Iteration #182 *****
Loss: 1223.777534
Feature norm: 68.794512
Error norm: 164.053764
Active features: 5787
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #183 *****
Loss: 1223.470322
Feature norm: 68.785173
Error norm: 98.245340
Active features: 5775
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #184 *****
Loss: 1223.307483
Feature norm: 68.869369
Error norm: 143.779608
Active features: 5775
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.030

***** Iteration #185 *****
Loss: 1223.034584
Feature norm: 68.860213
Error norm: 104.631207
Active features: 5778
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.034

***** Iteration #186 *****
Loss: 1222.911123
Feature norm: 68.947783
Error norm: 155.632960
Active features: 5767
Line search trials: 1
Line search step: 1.000000
Seconds re

In [29]:
print(len(X_test))

541


In [27]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

二 (O)
诊 (O)
（ (O)
1 (O)
9 (O)
7 (O)
6 (O)
年 (O)
8 (O)
月 (O)
7 (O)
日 (O)
) (O)
畏 (O)
寒 (O)
肢 (O)
冷 (O)
明 (O)
显 (O)
好 (O)
转 (O)
， (O)
能 (O)
外 (O)
出 (O)
散 (O)
步 (O)
而 (O)
不 (O)
怕 (O)
风 (O)
， (O)
四 (O)
肢 (O)
关 (O)
节 (O)
疼 (O)
痛 (O)
痊 (O)
愈 (O)
， (O)
唯 (O)
纳 (O)
少 (O)
恶 (O)
心 (O)
， (O)
舌 (O)
淡 (O)
， (O)
苔 (O)
薄 (O)
白 (O)
， (O)
脉 (O)
濡 (O)
等 (O)


In [28]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.98      0.98      0.98     16874
      B-SYM       0.54      0.47      0.51       236
      I-SYM       0.41      0.40      0.41       225
      E-SYM       0.57      0.50      0.53       236

avg / total       0.96      0.96      0.96     17571

